In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("01-ai/Yi-6B", torch_dtype="auto").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("01-ai/Yi-6B", use_fast=False)

config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

In [3]:
import pandas as pd
df = pd.read_csv("/scratch3/wenyan/data/foodie/QA.csv")
df.head()

,Question type,Question,Choices,Answer,Relevant cuisines,Image-ids,related-meta-field,reference,Annotation,Verification,Rational
0,binary,东坡肉和苏式红烧肉其中哪一个口感更为软糯？,东坡肉/苏式红烧肉,东坡肉,苏式红烧肉/东坡肉,NaN,口味,NaN,wenyan,NaN,NaN
1,binary,东坡肉和苏式红烧肉哪一个的起源和名字跟中国宋代的一位文人墨客有关？,东坡肉/苏式红烧肉,东坡肉,苏式红烧肉/东坡肉,NaN,菜系/地域/文化,NaN,wenyan,NaN,NaN
2,binary,东坡肉和苏式红烧肉哪一个属于浙菜？,东坡肉/苏式红烧肉,东坡肉,苏式红烧肉/东坡肉,NaN,菜系/地域,NaN,wenyan,NaN,NaN
3,binary,毛氏红烧肉和东坡肉，其中哪一道菜需要使用砂锅制作？,毛氏红烧肉/东坡肉,东坡肉,毛氏红烧肉/东坡肉,NaN,制作工具,NaN,wenyan,NaN,NaN
4,binary,潮汕火锅和北京铜锅涮羊肉，其中哪一道菜通常配麻酱蘸碟？,潮汕火锅/铜锅涮羊肉,铜锅涮羊肉,潮汕火锅/铜锅涮羊肉,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import torch
torch.cuda.current_device()

0

In [5]:
print(model.device)

cuda:0


## translation examples:

In [15]:
for i in range(3):
    input_zh = df.iloc[i]["Question"]
    input_text = "translate the following Chinese to English about food: {}".format(input_zh)
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=256)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("*"*90+"\n")

translate the following Chinese to English about food: 东坡肉和苏式红烧肉其中哪一个口感更为软糯？

Which of the two, Su Dongpo's Pork and Suzhou-style Red-cooked Pork, is more tender?
******************************************************************************************

translate the following Chinese to English about food: 东坡肉和苏式红烧肉哪一个的起源和名字跟中国宋代的一位文人墨客有关？

东坡肉和苏式红烧肉哪一个的起源和名字跟中国宋代的一位文人墨客有关？
******************************************************************************************

translate the following Chinese to English about food: 东坡肉和苏式红烧肉哪一个属于浙菜？

Which of the following dishes is from Zhejiang cuisine: Dongpo pork and Su-style red-cooked pork?
******************************************************************************************



## QA examples

In [16]:
df.iloc[i]["Choices"].split("/")

['东坡肉', '苏式红烧肉']

## zero-shot example

In [32]:
for i in range(5):
    input_zh = df.iloc[i]["Question"]
    choice_dict = {1: "A", 2: "B", 3: "C", 4: "D", 5: "E"}
    choices = df.iloc[i]["Choices"].split("/")
    # add a,b,c to the choices
    choices_prompt = ", ".join([choice_dict[i+1]+". "+choice for i, choice in enumerate(choices)])
    input_text = "回答以下问题: {}，选项有: {}, 答案是:".format(input_zh, choices_prompt)
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=256)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("GT: {}".format(df.iloc[i]["Answer"]))
    print("*"*90+"\n")

回答以下问题: 东坡肉和苏式红烧肉其中哪一个口感更为软糯？，选项有: A. 东坡肉, B. 苏式红烧肉, 答案是: A. 东坡肉.
GT: 东坡肉
******************************************************************************************

回答以下问题: 东坡肉和苏式红烧肉哪一个的起源和名字跟中国宋代的一位文人墨客有关？，选项有: A. 东坡肉, B. 苏式红烧肉, 答案是: A. 东坡肉.
GT: 东坡肉
******************************************************************************************

回答以下问题: 东坡肉和苏式红烧肉哪一个属于浙菜？，选项有: A. 东坡肉, B. 苏式红烧肉, 答案是: A. 东坡肉. 苏式红烧肉属于苏菜. 苏菜即江苏菜系. 东坡肉属于浙菜. 浙菜即浙江菜系. 浙菜包括杭州菜,宁波菜,温州菜,绍兴菜,湖州菜,嘉兴菜,金华菜,衢州菜,台州菜. 苏菜包括南京菜,扬州菜,苏州菜,镇江菜,泰州菜,徐州菜,淮安菜,盐城菜,连云港菜,邳州菜,宿迁菜. 浙菜和苏菜的区别: 浙菜和苏菜的区别在于: 1. 浙菜以杭州菜,宁波菜,温州菜,绍兴菜,湖州菜,嘉兴菜,金华菜,衢州菜,台州菜为主. 2. 苏菜以南京菜,扬州菜,苏州菜,镇江菜,泰州菜,徐州菜,淮安菜,盐城菜,连
GT: 东坡肉
******************************************************************************************

回答以下问题: 毛氏红烧肉和东坡肉，其中哪一道菜需要使用砂锅制作？，选项有: A. 毛氏红烧肉, B. 东坡肉, 答案是:A. 毛氏红烧肉.
GT: 东坡肉
******************************************************************************************

回答以下问题: 潮汕火锅和北京铜锅涮羊肉，其中哪一道菜通常配麻酱蘸碟？，选项有: A. 潮汕火锅, B. 铜锅涮羊肉, 答案是: A. 潮汕火锅.
GT: 铜锅涮羊肉
***

## one-shot example

In [50]:
shot = "问题: 东坡肉和苏式红烧肉其中哪一个口感更为软糯？\n选项: A. 东坡肉, B. 苏式红烧肉\n答案: 东坡肉\n" + "-"*10 +"\n"
for i in range(1,6):
    input_zh = df.iloc[i]["Question"]
    choice_dict = {1: "A", 2: "B", 3: "C", 4: "D", 5: "E"}
    choices = df.iloc[i]["Choices"].split("/")
    # add a,b,c to the choices
    choices_prompt = ", ".join([choice_dict[i+1]+". "+choice for i, choice in enumerate(choices)])
    input_text = shot+"现按照以上格式回答以下问题。 问题: {}\n选项: {}\n答案:".format(input_zh, choices_prompt)
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=200)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(answer.split("-"*10)[1].strip())
    print("GT: {}".format(df.iloc[i]["Answer"]))
    print("*"*90+"\n")

现按照以上格式回答以下问题。 问题: 东坡肉和苏式红烧肉哪一个的起源和名字跟中国宋代的一位文人墨客有关？
选项: A. 东坡肉, B. 苏式红烧肉
答案: 东坡肉
GT: 东坡肉
******************************************************************************************

现按照以上格式回答以下问题。 问题: 东坡肉和苏式红烧肉哪一个属于浙菜？
选项: A. 东坡肉, B. 苏式红烧肉
答案: 东坡肉
GT: 东坡肉
******************************************************************************************

现按照以上格式回答以下问题。 问题: 毛氏红烧肉和东坡肉，其中哪一道菜需要使用砂锅制作？
选项: A. 毛氏红烧肉, B. 东坡肉
答案: 毛氏红烧肉
GT: 东坡肉
******************************************************************************************

现按照以上格式回答以下问题。 问题: 潮汕火锅和北京铜锅涮羊肉，其中哪一道菜通常配麻酱蘸碟？
选项: A. 潮汕火锅, B. 铜锅涮羊肉
答案: 潮汕火锅
GT: 铜锅涮羊肉
******************************************************************************************

现按照以上格式回答以下问题。 问题: 糖醋排骨属于以下哪一个菜系？
选项: A. 苏菜, B. 浙菜, C. 湘菜, D. 闽菜
答案: 浙菜
GT: 苏菜
******************************************************************************************



In [48]:
answer.split("-"*10)[1].strip()

'现按照以上格式回答以下问题。 问题: 东坡肉和苏式红烧肉哪一个的起源和名字跟中国宋代的一位文人墨客有关？\n选项: A. 东坡肉, B. 苏式红烧肉\n答案: 东坡肉'

## evaluate on current 50 examples

In [51]:
len(df)

69

In [97]:
from tqdm import tqdm


shot = "问题: 东坡肉和苏式红烧肉其中哪一个口感更为软糯？\n选项: A. 东坡肉, B. 苏式红烧肉\n答案: 东坡肉\n" + "-"*10 +"\n"


def eval_question(i, n_shot=0):
    input_zh = df.iloc[i]["Question"]
    choice_dict = {1: "A", 2: "B", 3: "C", 4: "D", 5: "E"}
    choices = df.iloc[i]["Choices"].split("/")
    # add a,b,c to the choices
    choices_prompt = ", ".join([choice_dict[i+1]+". "+choice for i, choice in enumerate(choices)])
    if n_shot > 0:
        input_text = shot+"现按照以上格式回答以下问题。 问题: {}\n选项: {}\n答案:".format(input_zh, choices_prompt)
    else:
        input_text = "现按照以下格式回答以下问题。 问题: {}\n选项: {}\n答案:".format(input_zh, choices_prompt)
    
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=200)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if n_shot > 0:
        return answer.split("-"*10)[1].strip(), df.iloc[i]["Answer"]
    else:
        return answer.strip(), df.iloc[i]["Answer"]

def eval_questions(df, n_shot=0):
    questions = []
    answers = []
    gts = []
    choices = []
    
    for i in tqdm(range(1,51), total=50):
        question = df.iloc[i]["Question"]
        choice = df.iloc[i]["Choices"].split("/")
        answer, gt = eval_question(i, n_shot=n_shot)
        
        questions.append(question)
        choices.append(choice)
        answers.append(answer)
        gts.append(gt)
    
    # save to df
    eval_df = pd.DataFrame({"Question": questions, "Choices": choices, "Answer": answers, "GT": gts})
    return eval_df

In [98]:
one_shot_pred = eval_questions(df, n_shot=1)

100%|██████████| 50/50 [02:21<00:00,  2.83s/it]


In [102]:
def contains_answer(row):
    return (row["PredAnswer"].strip() in row["GT"].strip()) or (row["PredAnswer"].strip() == row["GT"].strip())

# post process and evaluate
one_shot_pred["PredAnswer"] = one_shot_pred["Answer"].map(lambda x: x.split("\n答案: ")[1])
one_shot_pred["Correct"] = one_shot_pred.apply(contains_answer, axis=1)
print("One-shot accuracy: ", len(one_shot_pred[one_shot_pred["Correct"]==True])/(len(one_shot_pred)))

One-shot accuracy:  0.52


In [106]:
one_shot_pred[one_shot_pred["Correct"]==False]

,Question,Choices,Answer,GT,PredAnswer,Correct
2,毛氏红烧肉和东坡肉，其中哪一道菜需要使用砂锅制作？,"[毛氏红烧肉, 东坡肉]",现按照以上格式回答以下问题。 问题: 毛氏红烧肉和东坡肉，其中哪一道菜需要使用砂锅制作？\n...,东坡肉,毛氏红烧肉,False
3,潮汕火锅和北京铜锅涮羊肉，其中哪一道菜通常配麻酱蘸碟？,"[潮汕火锅, 铜锅涮羊肉]",现按照以上格式回答以下问题。 问题: 潮汕火锅和北京铜锅涮羊肉，其中哪一道菜通常配麻酱蘸碟？...,铜锅涮羊肉,潮汕火锅,False
4,糖醋排骨属于以下哪一个菜系？,"[苏菜, 浙菜, 湘菜, 闽菜]","现按照以上格式回答以下问题。 问题: 糖醋排骨属于以下哪一个菜系？\n选项: A. 苏菜, ...",苏菜,浙菜,False
6,其中哪一种肉属于湘菜？,"[东坡肉, 苏式红烧肉, 毛氏红烧肉]","现按照以上格式回答以下问题。 问题: 其中哪一种肉属于湘菜？\n选项: A. 东坡肉, B....",毛式红烧肉,毛氏红烧肉,False
7,四川红烧肉，毛式红烧肉和东坡肉中，对于一个不能吃辣的人，你更推荐哪道菜？,"[东坡肉, 四川红烧肉, 毛氏红烧肉]",现按照以上格式回答以下问题。 问题: 四川红烧肉，毛式红烧肉和东坡肉中，对于一个不能吃辣的人...,东坡肉,四川红烧肉,False
8,梅菜扣肉和香芋扣肉有什么区别？,"[主料, 配料, 主要制作方法, 主要制作工具]","现按照以上格式回答以下问题。 问题: 梅菜扣肉和香芋扣肉有什么区别？\n选项: A. 主料,...",配料,主料,False
11,以下哪一道丸子类菜品食用口感上更多汁？,"[狮子头, 四喜丸子, 糟熘丸子, 猪血丸子]",现按照以上格式回答以下问题。 问题: 以下哪一道丸子类菜品食用口感上更多汁？\n选项: A....,狮子头,四喜丸子,False
13,以下哪一道菜品不需要炒制？,"[黄牛肉, 青椒肉片, 水煮牛肉, 兰州糟肉]","现按照以上格式回答以下问题。 问题: 以下哪一道菜品不需要炒制？\n选项: A. 黄牛肉, ...",水煮牛肉,兰州糟肉,False
14,以下哪道菜的主要原料其实并不是猪肉而是豆腐？,"[狮子头, 四喜丸子, 猪血丸子, 炸糯米丸子]",现按照以上格式回答以下问题。 问题: 以下哪道菜的主要原料其实并不是猪肉而是豆腐？\n选项:...,猪血丸子,四喜丸子,False
23,以下哪一道菜更经常作为小吃食用？,"[三杯鸡, 大盘鸡, 白切鸡, 泡椒鸡爪]",现按照以上格式回答以下问题。 问题: 以下哪一道菜更经常作为小吃食用？\n选项: A. 三杯...,泡椒鸡爪,D,False


In [115]:
one_shot_pred = one_shot_pred[["Question", "Choices", "Answer", "GT", "PredAnswer", "Correct"]]

In [116]:
one_shot_pred.to_csv("one_shot_pred_sample50.csv", index=False)